## Step 3

In [7]:
import pandas as pd
import numpy as np

# 1. Load your modified dataset (which already has movement_mode and activity_hours)
df = pd.read_csv("../vessel_movements_dataset_modified.csv")

# --- STEP 3a: Calculate Maximum Speed (MS) ---
# MS = 1.066 x Vref
df['max_speed'] = 1.066 * df['vref']

# --- STEP 3b: Calculate Engine Load Factor (LF) ---
# LF = (Actual Speed / Max Speed)^3
df['load_factor'] = (df['speed_knots'] / df['max_speed']) ** 3

# Rule: Standard rounding to two decimal places
df['load_factor'] = df['load_factor'].round(2)

# Rule: If LF < 0.02 and mode is transit or maneuver, default LF to 0.02
# Note: We use the existing 'movement_mode' column from your CSV
mask = (df['load_factor'] < 0.02) & (df['movement_mode'].str.lower().isin(['transit', 'maneuver']))
df.loc[mask, 'load_factor'] = 0.02

# --- SAVE THE MODIFIED DATASET ---
# This appends the new columns to your existing modified CSV
df.to_csv("../vessel_movements_dataset_modified.csv", index=False)

print("Step 3 Calculations Complete! New columns 'max_speed' and 'load_factor' have been added.")
display(df[['vessel_id', 'speed_knots', 'vref', 'movement_mode', 'max_speed', 'load_factor']].head())

Step 3 Calculations Complete! New columns 'max_speed' and 'load_factor' have been added.


,vessel_id,speed_knots,vref,movement_mode,max_speed,load_factor
0,10498900,12.59,13.23,Maneuver,14.10318,0.71
1,10498900,12.47,13.23,Transit,14.10318,0.69
2,10498900,12.55,13.23,Transit,14.10318,0.70
3,10498900,12.47,13.23,Transit,14.10318,0.69
4,10498900,12.52,13.23,Transit,14.10318,0.70


## Step 5

In [ ]:
import pandas as pd
import numpy as np

# 1. Load your modified dataset and the LLAF lookup table
df = pd.read_csv('../vessel_movements_dataset_modified.csv')
llaf_df = pd.read_csv('../llaf_table.csv')
cf_df = pd.read_csv('../calculation_factors.xlsx.csv')

# Clean column names (remove any leading/trailing spaces)
df.columns = [c.strip() for c in df.columns]
cf_df.columns = [c.strip() for c in cf_df.columns]

# --- STEP 5a: Low Load Adjustment Factor (LLAF) Lookup ---
# Rule: Multiply LF by 100, round to nearest integer
df['percent_lf'] = (df['load_factor'] * 100).round().astype(int)

# Rule: If %LF < 2% and mode is transit or maneuver, default to 2%
mask_5a = (df['percent_lf'] < 2) & (df['movement_mode'].str.lower().isin(['transit', 'maneuver']))
df.loc[mask_5a, 'percent_lf'] = 2

# Rule: LLAF will be 1 if %LF is more than 20%
# Prep the lookup table
llaf_lookup = llaf_df.set_index(llaf_df['Load'].str.replace('%', '').astype(int))

def get_llaf_factors(row):
    pct = row['percent_lf']
    if pct > 20:
        # All factors are 1.0 for loads above 20%
        return pd.Series([1.0]*3, index=['llaf_CO2', 'llaf_CH4', 'llaf_N2O'])
    
    # Fetch values from table for specific load
    lookup_val = max(2, min(20, pct))
    res = llaf_lookup.loc[lookup_val]
    return pd.Series([res['CO2'], res['CH4'], res['N2O']], index=['llaf_CO2', 'llaf_CH4', 'llaf_N2O'])

df[['llaf_CO2', 'llaf_CH4', 'llaf_N2O']] = df.apply(get_llaf_factors, axis=1)

# --- STEP 5b & 5c: Emission Calculations ---
# Formula: Emission = Fuel_Consumption * Cf * LLAF
# Prep Carbon Factor maps
cf_maps = {
    'CO2': cf_df.set_index('Fuel Type')['Cf_CO2'].to_dict(),
    'CH4': cf_df.set_index('Fuel Type')['Cf_CH4'].to_dict(),
    'N2O': cf_df.set_index('Fuel Type')['Cf_N2O'].to_dict()
}

def calculate_mass_emissions(row):
    # Retrieve Carbon Factors for each engine's fuel
    cf_me = {p: cf_maps[p].get(row['main_engine_fuel_type'], 0) for p in ['CO2', 'CH4', 'N2O']}
    cf_ae = {p: cf_maps[p].get(row['aux_engine_fuel_type'], 0) for p in ['CO2', 'CH4', 'N2O']}
    cf_ab = {p: cf_maps[p].get(row['boil_engine_fuel_type'], 0) for p in ['CO2', 'CH4', 'N2O']}
    
    # Main Engine (Applies LLAF)
    # Using your friend's 'me_fual_consumption(tonnes)' column
    e_me = {p: row['me_fual_consumption(tonnes)'] * cf_me[p] * row[f'llaf_{p}'] for p in ['CO2', 'CH4', 'N2O']}
    
    # Auxiliary Engine and Boiler (LLAF = 1)
    e_ae = {p: row['ae_fual_consumption(tonnes)'] * cf_ae[p] for p in ['CO2', 'CH4', 'N2O']}
    e_ab = {p: row['ab_fual_consumption(tonnes)'] * cf_ab[p] for p in ['CO2', 'CH4', 'N2O']}
    
    # Total mass for each gas
    total = {p: e_me[p] + e_ae[p] + e_ab[p] for p in ['CO2', 'CH4', 'N2O']}
    
    # Calculate CO2 Equivalent (GWP: CH4=28, N2O=265)
    co2_eq = total['CO2'] + (total['CH4'] * 28) + (total['N2O'] * 265)
    
    return pd.Series([total['CO2'], total['CH4'], total['N2O'], co2_eq], 
                     index=['total_co2', 'total_ch4', 'total_n2o', 'total_co2_eq'])

# Apply calculation
df[['total_co2', 'total_ch4', 'total_n2o', 'total_co2_eq']] = df.apply(calculate_mass_emissions, axis=1)

# Save the updated dataset
df.to_csv('../vessel_movements_dataset_modified.csv', index=False)
print("Step 5 Complete! Emissions and CO2eq saved.")

FileNotFoundError: [Errno 2] No such file or directory: '../calculation_factors.xlsx - Cf.csv'